#**사용법**

##1. 사용 중인 부시나비 계정 및 비밀번호 입력 (글로벌 사이트 한정)
ex) bushinavi@email.com / ********

    
##2. 코드 실행
부시나비 사이트에서 대전기록을 다운로드합니다.
(소요시간 5분 ~ 10분)


In [ ]:
아이디 = '' #@param {type : 'string'}
비밀번호 = '' #@param {type:"string", password:true}
#@markdown - 부시노드에서 사용 중인 로그인 정보를 입력해주세요.

import shutil
import sys
import subprocess
import importlib.util

def run(cmd):
    subprocess.run(cmd, shell=True, check=True)

# ---------------- Chrome 설치 체크 ----------------
if shutil.which("google-chrome") is None:
    print("[INFO] google-chrome 설치 중")
    run("wget -q -O - https://dl.google.com/linux/linux_signing_key.pub | sudo apt-key add -")
    run('echo "deb [arch=amd64] http://dl.google.com/linux/chrome/deb/ stable main" | sudo tee /etc/apt/sources.list.d/google-chrome.list')
    run("sudo apt-get update")
    run("sudo apt-get install -y google-chrome-stable")

def pip_install(package):
    run(f"python -m pip install {package}")

# selenium 체크
if importlib.util.find_spec("selenium") is None:
    pip_install("selenium")

# webdriver-manager 체크
if importlib.util.find_spec("webdriver_manager") is None:
    pip_install("webdriver-manager")

import selenium as selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from datetime import datetime
from google.colab import files
import pandas as pd
import time
import csv
import re

# ---------------- 기본 설정 ----------------
BASE_URL = "https://www.en.bushi-navi.com"

EMAIL = 아이디       # ← 여기에 이메일
PASSWORD = 비밀번호 # ← 여기에 비밀번호

LIMIT = 100    # 페이지당 이벤트 수, 20, 50, 100으로만 설정 가능
SLEEP_PAGE = 0.8   # 페이지 전환 간 대기
SLEEP_EVENT = 0.8   # 이벤트 페이지 간 대기
SLEEP_HISTORY = 0.8  # history 페이지 간 대기

OUTPUT_XLSX = "bushinavi_history.xlsx"

# ---------------- 브라우저 관리 ----------------
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
# 필요하면 백그라운드 실행:
options.add_argument("--headless=new") # Uncommented and added
options.add_argument("--no-sandbox") # Added for Colab compatibility
options.add_argument("--disable-dev-shm-usage") # Added for Colab compatibility

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
wait = WebDriverWait(driver, 15)

# ---------------- 쿠키 방지 ----------------
def accept_cookies_if_present():
    try:
        btn = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((
                By.ID,
                "CybotCookiebotDialogBodyLevelButtonLevelOptinAllowAll"
            ))
        )
        driver.execute_script("arguments[0].click();", btn)

        # 쿠키 팝업이 완전히 사라질 때까지 대기
        WebDriverWait(driver, 5).until(
            EC.invisibility_of_element_located((
                By.ID,
                "CybotCookiebotDialog"
            ))
        )

        print("[INFO] 쿠키 동의 팝업 처리 완료")
        time.sleep(0.5)

    except Exception:
        print("[INFO] 쿠키 팝업 없음")


# ---------------- 로그인 ----------------
def login():
    login_url = f"{BASE_URL}/my_event?limit={LIMIT}&offset=0&past_event_display_flg=1"
    driver.get(login_url)

    accept_cookies_if_present()

    # 로그인 폼 찾기
    mail_input = wait.until(EC.presence_of_element_located((By.NAME, "mail")))
    pass_input = driver.find_element(By.NAME, "pass")

    mail_input.clear()
    mail_input.send_keys(EMAIL)

    pass_input.clear()
    pass_input.send_keys(PASSWORD)

    login_btn = driver.find_element(By.CLASS_NAME, "blue-btn-01")
    login_btn.click()

    # 로그인 후 my_event 페이지가 뜰 때까지 대기
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    time.sleep(2)

# ---------------- 이벤트 ID 수집 ----------------
def collect_event_ids():
    # 페이지 돌면서 모든 이벤트 ID 수집
    event_ids = []
    offset = 0

    while True:
        collect_id_url = f"{BASE_URL}/my_event?limit={LIMIT}&offset={offset}&past_event_display_flg=1"
        print(f"[INFO] 이벤트 목록 페이지 접속")
        driver.get(collect_id_url)
        time.sleep(SLEEP_PAGE)

        wrappers = driver.find_elements(By.CLASS_NAME, "myd-wrapper")

        # 더 이상 이벤트가 없으면 종료
        if not wrappers:
            print("[INFO] 이벤트 모두 찾음")
            break

        print(f"[INFO] 이 페이지에서 찾은 myd-wrapper 개수: {len(wrappers)}")

        for wrapper in wrappers:
            try:
                title_area = wrapper.find_element(By.CLASS_NAME, "myd-team-info-title-wrapper")
                apply_btn = title_area.find_element(By.CLASS_NAME, "btn-apply")
                detail_url = apply_btn.get_attribute("href")
                # 예: https://www.en.bushi-navi.com/my_event/123456
                if detail_url and "/my_event/" in detail_url:
                    event_id = detail_url.split("/my_event/")[-1].split("/")[0]
                    if event_id.isdigit():
                        event_ids.append(event_id)
            except Exception as e:
                print(f"[DEBUG] 스킵: {e}")
                continue


        offset += LIMIT

    # 순서 유지하면서 중복 제거 및 과거 순으로 정렬
    ## 최신 순으로 변경하고 싶다면 코드 뒤 [::-1] 부분 제거
    unique_ids = list(dict.fromkeys(event_ids))[::-1]
    print(f"[INFO] 총 수집된 이벤트 ID 수: {len(unique_ids)}")
    return unique_ids

# ---------------- 이벤트 정보(대회일, 샵 이름) 수집 ----------------
def get_event_meta(event_id):
    print(f"[INFO] 이벤트 정보 크롤링 시작")
    print(f"수집중 ...")
    meta_url = f"{BASE_URL}/my_event/{event_id}"
    driver.get(meta_url)
    time.sleep(SLEEP_EVENT)

    place = driver.find_element(
        By.CLASS_NAME, "description-shop-name"
    ).text.strip()

    date_area = driver.find_element(By.CLASS_NAME, "myd-event-info-list-ul-01")
    date_text = date_area.find_element(
        By.CLASS_NAME, "myd-event-info-list-ul-item-02"
    ).text.strip()

    # May-06-2025 7:30 PM (KST) 날짜 형식 2025-03-06 형식으로 변경
    event_date = datetime.strptime(date_text.split(" (")[0], "%b-%d-%Y %I:%M %p")
    return event_date.strftime("%Y-%m-%d"), place

# ---------------- 대전 기록(라운드 수, 승패유무 등) 수집 ----------------
def get_event_history(event_ids):
    rows = []
    print(f"[INFO] 대전기록 크롤링 시작")
    print(f"수집중 ...")

    for idx, event_id in enumerate(event_ids, start=1):
        history_url = f"{BASE_URL}/event/{event_id}/history"
        driver.get(history_url)

        try:
            tbody = wait.until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody"))
            )
        except Exception:
            print(f"[WARN] 스킵 : 대전기록이 없는 이벤트")
            time.sleep(SLEEP_HISTORY)
            continue

        for tr in tbody.find_elements(By.TAG_NAME, "tr"):
            tds = tr.find_elements(By.TAG_NAME, "td")
            if len(tds) < 3:
                continue

            """ 오류시 아래 주석 해체 """
            opponent = tds[1].text.split("\n")[0].strip()
            result = tds[2].text.strip()

            if re.fullmatch(r"B\d{4}", opponent):
                continue

            rows.append((event_id, opponent, result))

            """
            cell_values = [td.text.strip() for td in tds]
            if any(cell_values):  # 완전히 빈 행은 스킵
                continue

            # ---- 닉네임 정리 (B0001) 형식 제외 ----
            opponent_raw = cell_values[1]
            opponent = opponent_raw.split("\n")[0].strip()

            if re.fullmatch(r"B\d{4}", opponent):
                continue

            cell_values[1] = opponent
            rows.append([event_id] + cell_values)
            """

        time.sleep(SLEEP_HISTORY)
    return rows


def main():
    if not EMAIL or not PASSWORD:
      raise ValueError("아이디 또는 비밀번호가 입력되지 않았습니다.")

    try:
        login()
        event_ids = collect_event_ids()

        # 메타
        meta = {eid: get_event_meta(eid) for eid in event_ids}

        # 히스토리
        history = get_event_history(event_ids)

        match_rows = []
        stat_rows = []
        last_event = None

        for event_id, opponent, result in history:
            date, place = meta[event_id]

            A = date if event_id != last_event else ""
            B = place if event_id != last_event else ""
            C = D = E = ""

            if opponent == "Won by Default":
                D = "부전"
                stat_rows.append((opponent, 1, 0, 0))
            elif "Double Loss" in result:
                D = "쌍방"
                E = opponent
                stat_rows.append((opponent, 0, 0, 1))
            else:
                E = opponent
                if result == "1 - 0":
                    C = 1
                    stat_rows.append((opponent, 1, 0, 0))
                elif result == "0 - 1":
                    C = 0
                    stat_rows.append((opponent, 0, 1, 0))

            match_rows.append([A, B, C, D, E])
            last_event = event_id

        # ---- 통계 ----
        df_stat = pd.DataFrame(
            stat_rows, columns=["대전상대", "승", "패", "쌍방"]
        )

        stat = df_stat.groupby("대전상대").sum().reset_index()
        stat["승률"] = (
            stat["승"] / (stat["승"] + stat["패"])
        ).fillna(0).round(2)

        stat = stat[["대전상대", "승", "패", "승률", "쌍방"]]

        # ---- 엑셀 출력 ----
        df_match = pd.DataFrame(
            match_rows,
            columns=["대회일", "대회장소", "승패", "비고", "대전상대"]
        )

        with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
            stat.to_excel(writer, sheet_name="상대전적", index=False)
            df_match.to_excel(writer, sheet_name="대전기록", index=False)

        print(f"[완료] {OUTPUT_XLSX} 생성")

        print("파일 다운로드 중")
        files.download(OUTPUT_XLSX)

    finally:
        driver.quit()


if __name__ == "__main__":
    main()